# Data Preparation

This notebook handles the preparation of the reference solution data used in the experiments of the paper. Among them, some are loaded from `.mat` files acquired from the [JaxPI repository](https://github.com/PredictiveIntelligenceLab/jaxpi/tree/pirate/examples), while others are obtained through knowledge of the equation's analytic solution.

In [ ]:
import os

import scipy
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
data_dir = os.path.join(parent_dir, "data")

os.makedirs(data_dir, exist_ok=True)

plots_dir = os.path.join(parent_dir, "plots")

os.makedirs(plots_dir, exist_ok=True)

## Auxiliaries

In [ ]:
# Function to save .mat files as .npz files for uniformity
def mat_to_np(filename):
    
    data = scipy.io.loadmat(f'mat/{filename}.mat')
    
    filtered_data = {key: data[key] for key in ['t', 'x', 'usol']}

    npz_filename = os.path.join(data_dir, filename + ".npz")

    np.savez_compressed(npz_filename, **filtered_data)

    print(f"Successfully saved file {filename}.")

In [ ]:
# Function that receives a function to create a reference solution
def create_sol(points, func, filename):

    x1, x2 = tuple(points.values())
    X1, X2 = np.meshgrid(x1, x2, indexing='ij')
    coords = np.stack([X1.flatten(), X2.flatten()], axis=1)
    
    N_1, N_2 = x1.shape[0], x2.shape[0]

    ref = dict()
    for key in points:        
        ref[key] = points[key].reshape(-1,1).T
    
    result = func(coords)
    
    ref['usol'] = result.reshape((N_1, N_2))

    npz_filename = os.path.join(data_dir, filename + ".npz")

    np.savez_compressed(npz_filename, **ref)

    print(f"Successfully saved file {filename}.")

In [ ]:
LABEL_FS = 14
TITLE_FS = 16
TICK_FS = 12
CBAR_FS = 14

# Function to plot the reference solution to an equation
def plot_sol(ref, eqname, save_fig=False, clim=None):
    solplot = ref['usol']

    if 't' in ref.files:
        t = ref['t'].squeeze()
        y = None
    else:
        t = None
        y = ref['y'].squeeze()
        
    x = ref['x'].squeeze()

    if t is not None:
        extent = [t.min(), t.max(), x.min(), x.max()]
    else:
        extent = [x.min(), x.max(), y.min(), y.max()]

    plt.figure(figsize=(6,4.5))
    cmap = sns.color_palette("Spectral", as_cmap=True)#.reversed()
    img = plt.imshow(solplot.T, aspect='auto', origin='lower', extent=extent, cmap=cmap)
    if clim is not None:
        img.set_clim(*clim)
    
    #cbar = plt.colorbar()
    colorbar_label = r"$u(t,x)$" if t is not None else r"$u(x,y)$"

    # Create horizontal colorbar above the plot
    cbar = plt.colorbar(orientation='horizontal', pad=0.2, fraction=0.05, aspect=30)
    cbar.ax.tick_params(labelsize=CBAR_FS)
    cbar.set_label(colorbar_label, fontsize=LABEL_FS, labelpad=5)
    cbar.ax.xaxis.set_label_position('bottom')

    
    plt.xlabel(r"$t$" if t is not None else r"$x$", fontsize=LABEL_FS)
    plt.ylabel(r"$x$" if t is not None else r"$y$", fontsize=LABEL_FS)
    plt.title(f"Solution to {eqname} Equation", fontsize=TITLE_FS)

    plt.tick_params(axis='both', which='major', labelsize=TICK_FS)

    if save_fig:
        plots_dir = os.path.join(parent_dir, "plots")
        plt.savefig(f"{plots_dir}/app_{eqname.lower()}.pdf", format="pdf", bbox_inches="tight")
    
    plt.show()

## Equations

### 1. Allen-Cahn

The Allen-Cahn equation for $t \in \left[0,1\right],~ x \in \left[-1,1\right]$ is given by

$$
    \frac{\partial u}{\partial t} - 10^{-4}\frac{\partial^2u}{\partial x^2} = 5\left(u-u^3\right),
$$

subject to initial/boundary conditions:

$$ u\left(0,x\right) = x^2 \cos\left(\pi x\right), $$
$$    u\left(t,-1\right) = u\left(t,1\right), $$
$$    \frac{\partial u}{\partial x}\left(t,-1\right) = \frac{\partial u}{\partial x}\left(t,1\right). $$

In [ ]:
filename = 'allen_cahn'

# Save as npz
mat_to_np(filename)

In [ ]:
filename = 'allen_cahn'

# Load file and display for documentation purposes
ref = np.load(os.path.join(data_dir,filename+'.npz'))

plot_sol(ref, "Allen–Cahn", save_fig=True, clim=(-1,1))

### 2. Burgers

The Burgers equation for $t \in \left[0,1\right],~ x \in \left[-1,1\right]$ is given by

$$
    \frac{\partial u}{\partial t} + u\frac{\partial u}{\partial x} = \frac{1}{100\pi} \frac{\partial^2 u}{\partial x^2}, 
$$

subject to initial/boundary conditions:

$$ u\left(0,x\right) = -\sin\left(\pi x\right), $$
$$    u\left(t,-1\right) = u\left(t,1\right) = 0. $$

In [ ]:
filename = 'burgers'

# Save as npz
mat_to_np(filename)

In [ ]:
filename = 'burgers'

# Load file and display for documentation purposes
ref = np.load(os.path.join(data_dir,filename+'.npz'))

plot_sol(ref, "Burgers'", save_fig=True, clim=(-1,1))

### 3. Korteweg-De Vries

The KdV equation for $t \in \left[0,1\right],~ x \in \left[-1,1\right]$ is given by

$$
    \frac{\partial u}{\partial t} + u\frac{\partial u}{\partial x} + 0.022^2 \frac{\partial^3 u}{\partial x^3} = 0,
$$

subject to initial/boundary conditions:

$$ u\left(0,x\right) = \cos\left(\pi x\right), $$
$$    u\left(t,-1\right) = u\left(t,1\right). $$

In [ ]:
filename = 'kdv'

# Save as npz
mat_to_np(filename)

In [ ]:
filename = 'kdv'

# Load file and display for documentation purposes
ref = np.load(os.path.join(data_dir,filename+'.npz'))

plot_sol(ref, "Korteweg–De Vries", save_fig=True)#, clim=(-1,2))

### 4. Sine Gordon

The Sine Gordon equation for $t \in \left[0,1\right],~ x \in \left[0,1\right]$ is given by

$$
    \frac{\partial^2 u}{\partial t^2} - \frac{\partial^2 u}{\partial x^2} + \sin u = 0, 
$$

subject to initial/boundary conditions:

$$ u\left(0,x\right) = \sin\left(\pi x\right) , $$
$$ u\left(t,0\right) = u\left(t,1\right) = 0. $$

The analytic solution of this equation is known:

$$ u\left(t,x\right) = \frac{1}{2}\left[\sin\left(\pi\left(x+t\right)\right) + \sin\left(\pi\left(x-t\right)\right)\right]. $$

In [ ]:
points = {'t' : np.linspace(0, 1, 201), 'x' : np.linspace(0, 1, 201)}

In [ ]:
filename = 'sg'

def func(coords):
    return 0.5*(np.sin(np.pi*(coords[:,[1]] + coords[:,[0]])) + np.sin(np.pi*(coords[:,[1]] - coords[:,[0]])))

create_sol(points, func, filename)

In [ ]:
filename = 'sg'

# Load file and display for documentation purposes
ref = np.load(os.path.join(data_dir, filename+'.npz'))

plot_sol(ref, "Sine Gordon", save_fig=True)

### 5. Advection

The Advection equation for $t \in \left[0,1\right],~ x \in \left[0,2\pi\right]$ is given by

$$
    \frac{\partial u}{\partial t} + 20\frac{\partial u}{\partial x} = 0, 
$$

subject to initial/boundary conditions:

$$ u\left(0,x\right) = \sin\left( x\right) , $$
$$ u\left(t,x\right) = u\left(t,x+2\pi\right). $$

The analytic solution of this equation is known:

$$ u\left(t,x\right) = \sin\left(\text{mod}\left(x - 20t, 2\pi\right)\right). $$

In [ ]:
points = {'t' : np.linspace(0, 1, 201), 'x' : np.linspace(0, 2*np.pi, 512)}

In [ ]:
filename = 'advection'

def func(coords):
    return np.sin(np.mod(coords[:,[1]] - 20 * coords[:,[0]], 2 * np.pi))

create_sol(points, func, filename)

In [ ]:
filename = 'advection'

# Load file and display for documentation purposes
ref = np.load(os.path.join(data_dir,filename+'.npz'))

plot_sol(ref, "Advection", save_fig=True)

### 6. Helmholtz

The Helmholtz equation for $x \in \left[-1,1\right],~ y \in \left[-1,1\right]$ is given by

$$
    \frac{\partial^2 u}{\partial x^2} + \frac{\partial^2 u}{\partial y^2} + u = \left[1 -\pi^2\left(a_1^2 + a_2^2\right)\right]\sin\left(a_1\pi x\right)\sin\left(a_2\pi y\right), 
$$

subject to boundary conditions:

$$ u\left(-1,y\right) = u\left(1,y\right) = u\left(x,-1\right) = u\left(x,1\right) = 0. $$

The analytical solution of this equation is known:

$$ u\left(x,y\right) = \sin\left(a_1\pi x\right)\sin\left(a_2\pi y\right). $$

In our experiments we solve the Helmholtz equation for $a_1 = 1$, $a_2 = 4$.

In [ ]:
points = {'x' : np.linspace(-1, 1, 512), 'y' : np.linspace(-1, 1, 512)}

In [ ]:
filename = 'helmholtz_1-4'

def func(coords):
    a1, a2 = 1.0, 4.0
    return np.sin(np.pi * a1 * coords[:, [0]]) * np.sin(np.pi * a2 * coords[:, [1]])

create_sol(points, func, filename)

In [ ]:
filename = 'helmholtz_1-4'

# Load file and display for documentation purposes
ref = np.load(os.path.join(data_dir,filename+'.npz'))

plot_sol(ref, r"Helmholtz", save_fig=True, clim=(-1,1))

### 7. Poisson

The Poisson equation for $x \in \left[-1,1\right],~ y \in \left[-1,1\right]$ is given by

$$
    \frac{\partial^2 u}{\partial x^2} + \frac{\partial^2 u}{\partial y^2} = -2\omega^2\pi^2\sin\left(\omega\pi x\right)\sin\left(\omega\pi y\right), 
$$

subject to boundary conditions:

$$ u\left(-1,y\right) = u\left(1,y\right) = u\left(x,-1\right) = u\left(x,1\right) = 0. $$

The analytical solution of this equation is known:

$$ u\left(x,y\right) = \sin\left(\omega\pi x\right)\sin\left(\omega\pi y\right). $$

In our experiments we solve the Poisson equation for different values of $\omega$, therefore we modify the code above used for plotting.

In [ ]:
LABEL_FS = 14
TITLE_FS = 16
TICK_FS = 12
CBAR_FS = 14

def plot_poisson_solutions(ref_list, omega_values, save_fig=False, clim=None):
    
    fig = plt.figure(figsize=(11, 4))
    gs = fig.add_gridspec(2, 3, height_ratios=[1, 0.05], wspace=0.4, hspace=0.3)
    
    axes = np.array([
        fig.add_subplot(gs[0, 0]), fig.add_subplot(gs[0, 1]), fig.add_subplot(gs[0, 2])
    ])
    
    cax = fig.add_subplot(gs[1, :])

    cmap = sns.color_palette("Spectral", as_cmap=True)

    img_list = []
    for ax, ref, omega in zip(axes.flatten(), ref_list, omega_values):
        solplot = ref['usol']
        x = ref['x'].squeeze()
        y = ref['y'].squeeze()
        extent = [x.min(), x.max(), y.min(), y.max()]
        img = ax.imshow(solplot.T, aspect='auto', origin='lower', extent=extent, cmap=cmap)
        if clim is not None:
            img.set_clim(*clim)
        ax.set_title(fr"$\omega = {omega}$", fontsize=TITLE_FS)
        ax.tick_params(axis='both', which='major', labelsize=TICK_FS)
        img_list.append(img)


    for ax in axes:
        ax.set_xlabel(r"$x$", fontsize=LABEL_FS)
        ax.set_ylabel(r"$y$", fontsize=LABEL_FS)


    # Shared colorbar on the right
    cbar = fig.colorbar(img_list[-1], cax=cax, orientation='horizontal')
    cbar.ax.tick_params(labelsize=CBAR_FS)
    cbar.set_label(r"$u(x,y)$", fontsize=LABEL_FS, labelpad=10)

    pos = cax.get_position()
    cax.set_position([pos.x0, pos.y0 - 0.08, pos.width, pos.height])

    plt.suptitle("Solution to Poisson Equation", fontsize=TITLE_FS, y=1.05)
    #plt.tight_layout()

    if save_fig:
        plots_dir = os.path.join(parent_dir, "plots")
        plt.savefig(f"{plots_dir}/app_poisson.pdf", format="pdf", bbox_inches="tight")
    
    plt.show()



Create the reference solutions:

In [ ]:
points = {'x' : np.linspace(-1, 1, 512), 'y' : np.linspace(-1, 1, 512)}

# w = 1
def func(coords):
    a1, a2 = 1.0, 1.0
    return np.sin(np.pi * a1 * coords[:, [0]]) * np.sin(np.pi * a2 * coords[:, [1]])

create_sol(points, func, 'poisson_1')

# w = 2
def func(coords):
    a1, a2 = 2.0, 2.0
    return np.sin(np.pi * a1 * coords[:, [0]]) * np.sin(np.pi * a2 * coords[:, [1]])

create_sol(points, func, 'poisson_2')

# w = 4
def func(coords):
    a1, a2 = 4.0, 4.0
    return np.sin(np.pi * a1 * coords[:, [0]]) * np.sin(np.pi * a2 * coords[:, [1]])

create_sol(points, func, 'poisson_4')

Plot the reference solutions:

In [ ]:
ref1 = np.load(os.path.join(data_dir,'poisson_1.npz'))
ref2 = np.load(os.path.join(data_dir,'poisson_2.npz'))
ref4 = np.load(os.path.join(data_dir,'poisson_4.npz'))

In [ ]:
omega_vals = [1, 2, 4]
refs = [ref1, ref2, ref4]

plot_poisson_solutions(refs, omega_vals, True, clim=(-1,1))